In [0]:
import pandas as pd
import pymssql
import pymysql
import pyspark.sql.functions as F

In [0]:
jdbcHostname = "54.245.193.229" #"54.245.193.229" 
jdbcDatabase = "DEV" #"KAGGLE" 
jdbcPort = "1433" #1433 
jdbcUsername = "sa"
jdbcPassword = "WindowsSQL-123"

host = "db4free.net"
db = "exocube"
uname = "sample_jenish"
pas = "Jenu7492@patel"

In [0]:
Source_Table_Name = "EMP_INFO"

In [0]:
def get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase):
    try:
        sql_connection = pymssql.connect(host=jdbcHostname,user=jdbcUsername,password=jdbcPassword,database=jdbcDatabase, as_dict=True, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymssql.OperationalError as e:
        print("connection failed")
    except:
        print("Unknown error occurred")
    else:
        return sql_connection

In [0]:
def get_myconnection(host, uname, pas, db):
    try:
        sql_myconnection = pymysql.connect(host=host,user=uname,passwd=pas,db=db, cursorclass=pymysql.cursors.DictCursor, autocommit=True)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymysql.OperationalError as e:
        if e.args[0] == 2003:
            print('server not found')
        elif e.args[0] == 1044:
            print('database not found')
        elif e.args[0] == 1045:
            print('username or password incorrect')
    except:
        print("Unknown error occurred")   
    else:
        return sql_myconnection

In [0]:
def display_mssql_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        cursor.execute(sql_disqry)
        sql_exec = cursor.fetchone()
        if len(sql_exec) > 0:
            sql_data = pd.DataFrame(cursor.fetchall())
            result = sql_data.head(n=None)
            display(result)
        else:
            print("No Record")
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
def display_target_data(sql_disqry, sql_conn):
    try:
            pds = spark.sql("(select * FROM {})".format(Source_Table_Name))
            pdf = pds.withColumn("Hash_Key",F.sha2(F.concat('Emp_Id','Emp_name', 'Designation', 'Department', 'City', 'Salary', F.unix_timestamp('Joining_Date')),256))
            pnd = pdf.toPandas()
            pd_df = pnd[['Emp_Id', 'Hash_Key']]
            display(pd_df)
    except pymssql.ProgrammingError as e:
        if e.args[0] == 208:
            print('table doesnt exist')
        elif e.args[0] == 102:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase)

In [0]:
sql_disqry ="(SELECT Emp_Id, lower(CONVERT(NVARCHAR(256), HASHBYTES('Sha2_256', CONCAT(Emp_Id, Emp_name, Designation, Department, City, Salary,DATEDIFF(s, '1970-01-01 00:00:00', Joining_Date))),2)) as Hash_Key FROM {})".format(Source_Table_Name)
display_mssql_data(sql_disqry, sql_conn)

Emp_Id Hash_Key 4 dfe448dc6fb5bdb1aeff670d6d0bbd564418613beb725841e803eb0a5ecb2833 6 440a2ad1a4472c9703dfeac22b8a6465004710701b5ece5b25b53aa8f8db7493 7 e1999a3a4b0815464685bdcf1e80bd9635ecf8e1394b3d8d2f692cd4a9024234 8 b5cd98edfd177e0cbf476728375c679147ba61a95cd62a89e919d7cb309525c1 9 bebd1156519378d61fced177c271841cfd30d3ef4af624a81c3e37c9344f42ef 11 12b200ab4af6e9c866382a8e0f0d0770c76e06704030b1feaaa0090e5598a165 12 b6525fd429191431da1120e2e89ab4c426915fdeefacc28d0fc83472a96b7d22 13 4f16df96196a8a2eb75c579ef792b961c2c000e47070264c157a3cea7bcf86ed 14 4367e01f192da2f38fb5612007ccdefacb5477b010077542a396fc4bb0432ba8 15 aaa762d2fbeb1d33dbe7f9bee3b9ac1d809d7b323fe94a73946fc09f54277964 17 6af61a0f1a02689a1c3102d02abd043cfde5a484787332fdb7d27630ba0f8091 18 bb925413f637d18fffa03c2f0657e1d2f96e29e6b05dcbc7d637a66df4d46c2f 19 6020045e47474171582b643c25122ccaed1176826f24f50f145b7c2f1d0aa091 20 872dc1ddc20f30fbc71518eb973862018c316284da6279175851da4450740eed 21 e592dde9589d27a3ce10145b5eb8bacd8c1b079a217361a59edd9c0f79c41963

In [0]:
display_target_data(sql_disqry, sql_conn)

Emp_Id Hash_Key 4 dfe448dc6fb5bdb1aeff670d6d0bbd564418613beb725841e803eb0a5ecb2833 6 440a2ad1a4472c9703dfeac22b8a6465004710701b5ece5b25b53aa8f8db7493 7 e1999a3a4b0815464685bdcf1e80bd9635ecf8e1394b3d8d2f692cd4a9024234 8 b5cd98edfd177e0cbf476728375c679147ba61a95cd62a89e919d7cb309525c1 9 bebd1156519378d61fced177c271841cfd30d3ef4af624a81c3e37c9344f42ef 11 12b200ab4af6e9c866382a8e0f0d0770c76e06704030b1feaaa0090e5598a165 12 b6525fd429191431da1120e2e89ab4c426915fdeefacc28d0fc83472a96b7d22 13 4f16df96196a8a2eb75c579ef792b961c2c000e47070264c157a3cea7bcf86ed 14 4367e01f192da2f38fb5612007ccdefacb5477b010077542a396fc4bb0432ba8 15 aaa762d2fbeb1d33dbe7f9bee3b9ac1d809d7b323fe94a73946fc09f54277964 17 6af61a0f1a02689a1c3102d02abd043cfde5a484787332fdb7d27630ba0f8091 18 bb925413f637d18fffa03c2f0657e1d2f96e29e6b05dcbc7d637a66df4d46c2f 19 6020045e47474171582b643c25122ccaed1176826f24f50f145b7c2f1d0aa091 20 872dc1ddc20f30fbc71518eb973862018c316284da6279175851da4450740eed 21 e592dde9589d27a3ce10145b5eb8bacd8c1b079a217361a59edd9c0f79c41963

In [0]:
def display_mysql_data(sql_disqry, sql_conn):
    try:
        cursor = sql_conn.cursor()
        sql_exec = cursor.execute(sql_disqry)
        if sql_exec:
            sql_data = pd.DataFrame(cursor.fetchall())
            result = sql_data.head(n=None)
            display(result)
        else:
            print(sql_exec)
            print("No Record")
    except pymysql.ProgrammingError as e:
        if e.args[0] == 1146:
            print('table doesnt exist')
        elif e.args[0] == 1064:
            print('sql query syntax error')  
    finally:
        sql_conn.close()

In [0]:
sql_conn = get_myconnection(host, uname, pas, db)

In [0]:
sql_disqry ="(select Emp_Id, sha2(CONCAT(Emp_Id, Emp_name, Designation, Department, City, Salary, UNIX_TIMESTAMP(Joining_Date)),256)as Hash_Key FROM {})".format(Source_Table_Name)
display_mysql_data(sql_disqry, sql_conn)

Emp_Id Hash_Key 4 dfe448dc6fb5bdb1aeff670d6d0bbd564418613beb725841e803eb0a5ecb2833 6 440a2ad1a4472c9703dfeac22b8a6465004710701b5ece5b25b53aa8f8db7493 7 e1999a3a4b0815464685bdcf1e80bd9635ecf8e1394b3d8d2f692cd4a9024234 8 b5cd98edfd177e0cbf476728375c679147ba61a95cd62a89e919d7cb309525c1 9 bebd1156519378d61fced177c271841cfd30d3ef4af624a81c3e37c9344f42ef 11 12b200ab4af6e9c866382a8e0f0d0770c76e06704030b1feaaa0090e5598a165 12 b6525fd429191431da1120e2e89ab4c426915fdeefacc28d0fc83472a96b7d22 13 4f16df96196a8a2eb75c579ef792b961c2c000e47070264c157a3cea7bcf86ed 14 4367e01f192da2f38fb5612007ccdefacb5477b010077542a396fc4bb0432ba8 15 aaa762d2fbeb1d33dbe7f9bee3b9ac1d809d7b323fe94a73946fc09f54277964 17 6af61a0f1a02689a1c3102d02abd043cfde5a484787332fdb7d27630ba0f8091 18 bb925413f637d18fffa03c2f0657e1d2f96e29e6b05dcbc7d637a66df4d46c2f 19 6020045e47474171582b643c25122ccaed1176826f24f50f145b7c2f1d0aa091 20 872dc1ddc20f30fbc71518eb973862018c316284da6279175851da4450740eed 21 e592dde9589d27a3ce10145b5eb8bacd8c1b079a217361a59edd9c0f79c41963